# Experiment 007: External Ensemble

Download and ensemble external pre-optimized solutions to find better configurations.

In [1]:
import math
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import time

getcontext().prec = 30
scale_factor = Decimal('1e15')

print("Libraries loaded")

Libraries loaded


In [2]:
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h
        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x * scale_factor), yoff=float(self.center_y * scale_factor))

    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
def get_side_length(trees):
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1]) / float(scale_factor)

def get_score(trees, n):
    if not trees:
        return 0.0
    side = get_side_length(trees)
    return side ** 2 / n

def has_collision(trees):
    if len(trees) <= 1:
        return False
    for i, tree1 in enumerate(trees):
        for j, tree2 in enumerate(trees):
            if i < j:
                if tree1.polygon.intersects(tree2.polygon) and not tree1.polygon.touches(tree2.polygon):
                    return True
    return False

def load_configuration_from_df(n, df):
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"])[1:] if str(row["x"]).startswith('s') else str(row["x"])
        y = str(row["y"])[1:] if str(row["y"]).startswith('s') else str(row["y"])
        deg = str(row["deg"])[1:] if str(row["deg"]).startswith('s') else str(row["deg"])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

print("Helper functions defined")

Helper functions defined


In [4]:
# Load our current best submission
print("Loading current best submission...")
our_df = pd.read_csv('/home/submission/submission.csv')

# Calculate our current score
our_scores = {}
our_total = 0
for n in range(1, 201):
    trees = load_configuration_from_df(n, our_df)
    if trees:
        score = get_score(trees, n)
        our_scores[n] = score
        our_total += score

print(f"Our current total score: {our_total:.6f}")

Loading current best submission...


Our current total score: 70.679449


In [5]:
# Load SmartManoj submission
print("\nLoading SmartManoj submission...")
sm_df = pd.read_csv('/home/code/experiments/007_external_ensemble/smartmanoj_submission.csv')

# Calculate SmartManoj scores and check for overlaps
sm_scores = {}
sm_overlaps = []
sm_total = 0
for n in range(1, 201):
    trees = load_configuration_from_df(n, sm_df)
    if trees:
        if has_collision(trees):
            sm_overlaps.append(n)
            sm_scores[n] = float('inf')  # Invalid due to overlap
        else:
            score = get_score(trees, n)
            sm_scores[n] = score
            sm_total += score

print(f"SmartManoj total score (valid only): {sm_total:.6f}")
print(f"SmartManoj overlapping N values: {len(sm_overlaps)}")
if sm_overlaps:
    print(f"  First 10 overlapping: {sm_overlaps[:10]}")


Loading SmartManoj submission...


SmartManoj total score (valid only): 70.743774
SmartManoj overlapping N values: 0


In [6]:
# Compare configurations and find improvements
print("\nComparing configurations...")

improvements = []
for n in range(1, 201):
    our_score = our_scores.get(n, float('inf'))
    sm_score = sm_scores.get(n, float('inf'))
    
    if sm_score < our_score:
        improvement = our_score - sm_score
        improvements.append((n, our_score, sm_score, improvement))

print(f"\nFound {len(improvements)} N values where SmartManoj is better:")
for n, our, sm, imp in improvements[:20]:
    print(f"  N={n}: ours={our:.6f}, SM={sm:.6f}, improvement={imp:.6f}")

if improvements:
    total_improvement = sum(imp for _, _, _, imp in improvements)
    print(f"\nTotal potential improvement: {total_improvement:.6f} points")


Comparing configurations...

Found 2 N values where SmartManoj is better:
  N=77: ours=0.351174, SM=0.351114, improvement=0.000060
  N=134: ours=0.349761, SM=0.346475, improvement=0.003285

Total potential improvement: 0.003346 points


In [7]:
# Create ensemble - pick best configuration for each N
print("\nCreating ensemble...")

ensemble_configs = {}
ensemble_scores = {}
ensemble_sources = {}

for n in range(1, 201):
    our_score = our_scores.get(n, float('inf'))
    sm_score = sm_scores.get(n, float('inf'))
    
    if sm_score < our_score:
        # Use SmartManoj configuration
        trees = load_configuration_from_df(n, sm_df)
        ensemble_configs[n] = trees
        ensemble_scores[n] = sm_score
        ensemble_sources[n] = 'SmartManoj'
    else:
        # Use our configuration
        trees = load_configuration_from_df(n, our_df)
        ensemble_configs[n] = trees
        ensemble_scores[n] = our_score
        ensemble_sources[n] = 'Ours'

# Calculate ensemble total
ensemble_total = sum(ensemble_scores.values())
print(f"Ensemble total score: {ensemble_total:.6f}")
print(f"Improvement over our best: {our_total - ensemble_total:.6f} points")
print(f"Gap to target: {ensemble_total - 68.922808:.6f} points")

# Source distribution
source_counts = {}
for n, source in ensemble_sources.items():
    source_counts[source] = source_counts.get(source, 0) + 1
print(f"\nSource distribution:")
for source, count in source_counts.items():
    print(f"  {source}: {count} configurations")


Creating ensemble...


Ensemble total score: 70.676104
Improvement over our best: 0.003346 points
Gap to target: 1.753296 points

Source distribution:
  Ours: 198 configurations
  SmartManoj: 2 configurations


In [8]:
# Verify ensemble has no overlaps
print("\nVerifying ensemble has no overlaps...")
overlap_count = 0
for n in range(1, 201):
    if has_collision(ensemble_configs[n]):
        overlap_count += 1
        print(f"  N={n}: OVERLAP!")

print(f"Total overlaps: {overlap_count}")


Verifying ensemble has no overlaps...


Total overlaps: 0


In [9]:
# Save ensemble submission
print("\nSaving ensemble submission...")

rows = []
for n in range(1, 201):
    trees = ensemble_configs[n]
    for i, tree in enumerate(trees):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': f's{tree.center_x}',
            'y': f's{tree.center_y}',
            'deg': f's{tree.angle}'
        })

ensemble_df = pd.DataFrame(rows)
ensemble_df.to_csv('/home/code/experiments/007_external_ensemble/submission.csv', index=False)
ensemble_df.to_csv('/home/submission/submission.csv', index=False)

print(f"Saved ensemble submission with score {ensemble_total:.6f}")
print(f"Sample rows:")
print(ensemble_df.head())


Saving ensemble submission...
Saved ensemble submission with score 70.676104
Sample rows:
      id                       x                       y  \
0  001_0    s-48.196086194214246     s58.770984615214225   
1  002_0   s0.154097069621355887  s-0.038540742694794648   
2  002_1  s-0.154097069621372845  s-0.561459257305224058   
3  003_0      s1.123655816140301      s0.781101815992563   
4  003_1       s1.23405569584216      s1.275999500663759   

                       deg  
0                    s45.0  
1  s203.629377730656841550  
2   s23.629377730656791812  
3        s111.125132292893  
4         s66.370622269343  
